In [1]:
import os
import csv
import math
import random
import requests
from CitesphereConnectorTest import *
from CitesphereConnector import CitesphereConnector

In [2]:
auth_object = authObject()
auth_object.authType  = 'oauth'
auth_object.access_token="b7f4880b-7e33-4eca-b5a5-29c73adce87a"
connector = CitesphereConnector("https://diging-dev.asu.edu/citesphere-review/api", auth_object)
groups=connector.get_groups()

In [3]:
def downloadFile(file_id, citesphere_token):
    giles_url = f"https://diging.asu.edu/geco-giles-staging/api/v2/resources/files/{file_id}/content"
    folder_path = "citesphere-connector"
    os.makedirs(folder_path, exist_ok=True)
    filename = os.path.join(folder_path, f"{file_id}.pdf")
    headers = {
        "Authorization": f"Bearer {citesphere_token}",
        "Content-Type": "application/pdf;charset=UTF-8"
    }
    response = requests.get(giles_url, headers=headers)
    if response.status_code == 200:
        with open(filename, "wb") as file:
            file.write(response.content)
        return filename  
    else:
        return None  

def addToCSV(itm:list,csv_dict:dict):
    for value in itm["items"]:
        if value["key"] in csv_dict:
            continue
        value["path"] = ""
        if value["gilesUploads"]:
            giles_id = value["gilesUploads"][0]["uploadedFile"]["id"]
            if giles_id != "":
                value["path"] = downloadFile(giles_id, auth_object.access_token)
        csv_dict[value["key"]] = value

In [11]:
csv_dict={}
for group in groups:
    groupID = group["id"]
    collections = connector.get_collections(groupID)
    for x in collections["collections"]:
        numPages = math.ceil(x["numberOfItems"] / 50)
        if numPages == 0:
            continue

        if numPages == 1:
            itm = connector.get_collection_items_pg(groupID, x["key"], 1)
            addToCSV(itm,csv_dict)
        else:
            for i in range(numPages):
                itm = connector.get_collection_items_pg(groupID, x["key"], i + 1)
                addToCSV(itm,csv_dict)

PDF downloaded to: citesphere-connector/FILEQL8sKmrswS7p.pdf
PDF downloaded to: citesphere-connector/FILENf3i3D2v2XkG.pdf
PDF downloaded to: citesphere-connector/FILEcpf2PgsbX0YR.pdf
PDF downloaded to: citesphere-connector/FILEwUpCCo2Y5Tmx.pdf
Authentication failed with status code 500
FILEcHWoIjAxJSlV
PDF downloaded to: citesphere-connector/FILE3u01O8ugYrge.pdf
PDF downloaded to: citesphere-connector/FILESSuu8mSuZRdM.pdf
Authentication failed with status code 500
FILEfItViY7iUgLM


In [18]:
with open('citesphereCSV.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    key, val = random.choice(list(csv_dict.items()))
    fields = val.keys()
    csv_items = list(csv_dict.values())
    writer.writerow(fields)
    for i in range(len(csv_items)):
        writer.writerow(list(csv_items[i].values()))